In [32]:
#r "nuget: Microsoft.SemanticKernel, 1.33.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.33.0

In [33]:
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: Azure.Core, 1.44.1"



Installed Packages azure.core, 1.44.1 Azure.Identity, 1.13.1

In [34]:
using Azure.Core;
using Azure.Identity;

TokenCredential tokenCredential =
    new ChainedTokenCredential(
        new AzureCliCredential(),
        new VisualStudioCredential(),
        new DefaultAzureCredential(
            new DefaultAzureCredentialOptions()));

In [35]:
var tokenRequestContext = new TokenRequestContext(["499b84ac-1321-427f-aa17-267ca6975798/.default"], parentRequestId: null);
var accesstoken = await tokenCredential.GetTokenAsync(tokenRequestContext, System.Threading.CancellationToken.None);

In [36]:
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi.Extensions, 1.33.0-alpha"

Installed Packages Microsoft.SemanticKernel.Plugins.OpenApi.Extensions, 1.33.0-alpha

In [37]:
using Microsoft.SemanticKernel.Plugins.OpenApi;
using System.Net.Http.Headers;
#pragma warning disable SKEXP0040

KernelPlugin plugin = await OpenApiKernelPluginFactory.CreateFromOpenApiAsync(
    pluginName: "ADOPipelines",     
    filePath: "ado_pipelines_openapi.json",
    new OpenApiFunctionExecutionParameters()
    {
        ServerUrlOverride = new Uri("https://dev.azure.com/"),
        AuthCallback = (request, token) =>
        {
            request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", accesstoken.Token);
            Console.WriteLine($"Requesting {request.RequestUri} ");
            return Task.CompletedTask;
        } 
    }
);
kernel.Plugins.Add(plugin);

In [38]:
PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
var result = await kernel.InvokePromptAsync("give me the 3 first names of my ADO pipelines in my organization MY_ORGANIZATION in project MY_PROJECT api version 7.0", new(settings));
Console.WriteLine(result);

Requesting https://dev.azure.com/MY_ORGANIZATION/MY_PROJECT/_apis/pipelines?$top=3&api-version=7.0 
Here are the first 3 pipeline names in your organization "MY_ORGANIZATION" under the project "MY_PROJECT":

1. **(Deprecated) MD.Infra.System.ContainerImages.ACRTransfer.Retail**
2. **[Buddy]Mgmt.TenantMigrationJob.replication**
3. **[delete] InE.UnifiedPolicyConsumerService.Gated.Retail**
